In [1]:
import pandas as pd
import numpy as np
import os
# acquire
from env import host, user, password
from pydataset import data

# visualize
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(11, 9))
plt.rc('font', size=13)

# turn off pink warning boxes
import warnings
warnings.filterwarnings("ignore")

import acquire

# Exercises

## Acquire (acquire.py)
- Zillow
- For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.

- You will have a zillow.ipynb file and a helper file for each section in the pipeline.

- acquire & summarize




1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

    - Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
    - Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.
    - Only include properties that include a latitude and longitude value.
    


In [2]:
zillow_df= acquire.new_zillow_data()

In [3]:
zillow_df.head() #looking at first 5 values

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,buildingclasstypeid,buildingclassdesc,heatingorsystemtypeid,heatingorsystemdesc,buildingclasstypeid,buildingclassdesc,storytypeid,storydesc,typeconstructiontypeid,typeconstructiondesc
0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None
1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None
2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None
3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,NaN,None,2.0,Central,NaN,None,NaN,None,NaN,None
4,1970746,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,NaN,None,2.0,Central,NaN,None,NaN,None,NaN,None


In [4]:
#looking at shape before dropping duplicate columns
zillow_df.shape

(77579, 79)

In [5]:
#Looking at list of columns
zillow_df.columns.to_list()

['id',
 'parcelid',
 'airconditioningtypeid',
 'architecturalstyletypeid',
 'basementsqft',
 'bathroomcnt',
 'bedroomcnt',
 'buildingclasstypeid',
 'buildingqualitytypeid',
 'calculatedbathnbr',
 'decktypeid',
 'finishedfloor1squarefeet',
 'calculatedfinishedsquarefeet',
 'finishedsquarefeet12',
 'finishedsquarefeet13',
 'finishedsquarefeet15',
 'finishedsquarefeet50',
 'finishedsquarefeet6',
 'fips',
 'fireplacecnt',
 'fullbathcnt',
 'garagecarcnt',
 'garagetotalsqft',
 'hashottuborspa',
 'heatingorsystemtypeid',
 'latitude',
 'longitude',
 'lotsizesquarefeet',
 'poolcnt',
 'poolsizesum',
 'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
 'propertycountylandusecode',
 'propertylandusetypeid',
 'propertyzoningdesc',
 'rawcensustractandblock',
 'regionidcity',
 'regionidcounty',
 'regionidneighborhood',
 'regionidzip',
 'roomcnt',
 'storytypeid',
 'threequarterbathnbr',
 'typeconstructiontypeid',
 'unitcnt',
 'yardbuildingsqft17',
 'yardbuildingsqft26',
 'yearbuilt',
 'numberofstories',


In [6]:
#drop duplicate columns
zillow_df= zillow_df.loc[:, ~zillow_df.columns.duplicated()]

In [7]:
#verify duplicate columns were dropped (went from 79 to 68 columns)
zillow_df.shape

(77579, 68)

In [8]:
#making sure there is only one parcelid column
zillow_df['parcelid']

0        14297519
1        17052889
2        14186244
3        12177905
4        10887214
           ...   
77574    10833991
77575    11000655
77576    17239384
77577    12773139
77578    12826780
Name: parcelid, Length: 77579, dtype: int64

In [9]:
#making sure there is only one id column
zillow_df['id']

0        1727539
1        1387261
2          11677
3        2288172
4        1970746
          ...   
77574    2864704
77575     673515
77576    2968375
77577    1843709
77578    1187175
Name: id, Length: 77579, dtype: int64

In [23]:
#filter by parcel and max transaction date
zillow_df= zillow_df.groupby(by= 'parcelid').max('transaction')

def nulls_by_col(df):
    num_missing = df.isnull().sum()
    print(type(num_missing))
    rows = df.shape[0]
    prcnt_miss = num_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing': prcnt_miss})
    return cols_missing

def nulls_by_row(df):
    num_missing = df.isnull().sum(axis=1)
    prcnt_miss = num_missing / df.shape[1] * 100
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 'percent_cols_missing': prcnt_miss})\
    .reset_index()\
    .groupby(['num_cols_missing', 'percent_cols_missing']).count()\
    .rename(index=str, columns={'index': 'num_rows'}).reset_index()
    return rows_missing2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)


In [24]:
#functions
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    print(type(num_missing))
    rows = df.shape[0]
    prcnt_miss = num_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing': prcnt_miss})
    return cols_missing

def nulls_by_row(df):
    num_missing = df.isnull().sum(axis=1)
    prcnt_miss = num_missing / df.shape[1] * 100
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 'percent_cols_missing': prcnt_miss})\
    .reset_index()\
    .groupby(['num_cols_missing', 'percent_cols_missing']).count()\
    .rename(index=str, columns={'index': 'num_rows'}).reset_index()
    return rows_missing

In [25]:
#using functions
nulls_by_col(zillow_df)

<class 'pandas.core.series.Series'>


,num_rows_missing,percent_rows_missing
id,0,0.000000
airconditioningtypeid,52428,67.753066
architecturalstyletypeid,77175,99.733785
basementsqft,77331,99.935385
bathroomcnt,0,0.000000
bedroomcnt,0,0.000000
buildingclasstypeid,77366,99.980615
buildingqualitytypeid,27709,35.808532
calculatedbathnbr,609,0.787015
decktypeid,76767,99.206524


In [26]:
nulls_by_row(zillow_df)

,num_cols_missing,percent_cols_missing,parcelid
0,17,30.357142857142854,2
1,18,32.142857142857146,10
2,19,33.92857142857143,79
3,20,35.714285714285715,264
4,21,37.5,423
5,22,39.285714285714285,444
6,23,41.07142857142857,1194
7,24,42.857142857142854,7188
8,25,44.642857142857146,8686
9,26,46.42857142857143,11856


In [27]:
def summarize(df):
    '''
    summarize will take in a single argument (a pandas dataframe) 
    and output to console various statistics on said dataframe, including:
    # .head()
    # .info()
    # .describe()
    # value_counts()
    # observation of nulls in the dataframe
    '''
    print('=====================================================')
    print('Dataframe head: ')
    print(df.head(3))
    print('=====================================================')
    print('Dataframe info: ')
    print(df.info())
    print('=====================================================')
    print('Dataframe Description: ')
    print(df.describe())
    num_cols = [col for col in df.columns if df[col].dtype != 'O']
    cat_cols = [col for col in df.columns if col not in num_cols]
    print('=====================================================')
    print('DataFrame value counts: ')
    for col in df.columns:
        if col in cat_cols:
            print(df[col].value_counts())  
        else:
            print(df[col].value_counts(bins=10, sort=False))
            print('=====================================================')
            
    print('=====================================================')
    print('nulls in dataframe by column: ')
    print(nulls_by_col(df))
    print('=====================================================')
    print('nulls in dataframe by row: ')
    print(nulls_by_row(df))
    print('=====================================================')

In [28]:
summarize(zillow_df)

Dataframe head: 
               id  airconditioningtypeid  architecturalstyletypeid  \
parcelid                                                             
10711855  1087254                    NaN                       NaN   
10711877  1072280                    1.0                       NaN   
10711888  1340933                    1.0                       NaN   

          basementsqft  bathroomcnt  bedroomcnt  buildingclasstypeid  \
parcelid                                                               
10711855           NaN          2.0         3.0                  NaN   
10711877           NaN          2.0         4.0                  NaN   
10711888           NaN          2.0         4.0                  NaN   

          buildingqualitytypeid  calculatedbathnbr  decktypeid  ...  \
parcelid                                                        ...   
10711855                    8.0                2.0         NaN  ...   
10711877                    8.0                2.0        

In [29]:
#Looking at shape
zillow_df.shape

(77381, 56)

3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values

In [30]:
def miss_dup_values(df):
    '''
    this function takes a dataframe as input and will output metrics for missing values and duplicated rows,
    and the percent of that column that has missing values and duplicated rows
    '''
        # Total missing values
    mis_val = df.isnull().sum()
        # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)
        #total of duplicated
    dup = df.duplicated().sum()
        # Percentage of missing values
    dup_percent = 100 * dup / len(df)
        # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
    mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
        # Print some summary information
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
           "There are " + str(mis_val_table_ren_columns.shape[0]) +
           " columns that have missing values.")
    print( "  ")
    print (f"** There are {dup} duplicate rows that represents {round(dup_percent, 2)}% of total Values**")
        # Return the dataframe with missing information
    return mis_val_table_ren_columns

In [31]:
miss_dup_values(zillow_df)

Your selected dataframe has 56 columns.
There are 44 columns that have missing values.
  
** There are 0 duplicate rows that represents 0.0% of total Values**


,Missing Values,% of Total Values
buildingclasstypeid,77366,100.0
finishedsquarefeet13,77340,99.9
basementsqft,77331,99.9
storytypeid,77331,99.9
yardbuildingsqft26,77311,99.9
fireplaceflag,77209,99.8
architecturalstyletypeid,77175,99.7
typeconstructiontypeid,77159,99.7
finishedsquarefeet6,76995,99.5
pooltypeid10,76917,99.4


4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [33]:
miss_dup_values(zillow_df)

Your selected dataframe has 56 columns.
There are 44 columns that have missing values.
  
** There are 0 duplicate rows that represents 0.0% of total Values**


,Missing Values,% of Total Values
buildingclasstypeid,77366,100.0
finishedsquarefeet13,77340,99.9
basementsqft,77331,99.9
storytypeid,77331,99.9
yardbuildingsqft26,77311,99.9
fireplaceflag,77209,99.8
architecturalstyletypeid,77175,99.7
typeconstructiontypeid,77159,99.7
finishedsquarefeet6,76995,99.5
pooltypeid10,76917,99.4


- Prepare

1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.


2. Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

- The input:
- A dataframe
- A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
- A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
- The output:
    - The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
- hint:
    - Look up the dropna documentation.
    - You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
    - Make use of inplace, i.e. inplace=True/False.
     


3. Decide how to handle the remaining missing values:

    - Fill with constant value.
    - Impute with mean, median, mode.
    - Drop row/column

- wrangle_zillow.py
     - Functions of the work above needed to acquire and prepare a new sample of data.
     
     

## Mall Customers
### notebook
1. Acquire data from mall_customers.customers in mysql database.
2. Summarize data (include distributions and descriptive statistics).
3. Detect outliers using IQR.
4. Split data (train, validate, and test split).
5. Encode categorical columns using a one hot encoder (pd.get_dummies).
6. Handles missing values.
7. Scaling

- wrangle_mall.py
1. Acquire data from mall_customers.customers in mysql database.
2. Split the data into train, validate, and split
3. One-hot-encoding (pd.get_dummies)
4. Missing values
5. Scaling